In [4]:
import pandas as pd

df = pd.read_parquet("data/intermediate/m3-monthly_scaled.parquet")

In [5]:
horizon = 12  # <-- FORECAST HORIZON
input_size = horizon * 5

In [16]:
# Step 1: Extract time series windows


def extract_series(df_):
    if len(df_) < input_size + horizon:
        return None
    df_ = df_.sort_values("ds")
    return pd.Series(
        {
            "X": df_["y_scaled"].iloc[-(input_size + horizon) : -horizon].values.astype("float32"),
            "y": df_["y_scaled"].iloc[-horizon:].values.astype("float32"),
            "ds": df_["ds"].iloc[-horizon:].values,
        }
    )


infer_df = df.groupby("unique_id").apply(extract_series).dropna()
infer_df.to_parquet("data/intermediate/m3-monthly_inference_preprocessed.parquet")

/tmp/ipykernel_166908/652466577.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  infer_df = df.groupby("unique_id").apply(extract_series).dropna()


In [15]:
import numpy as np
import torch

device = "cuda"
X_scaled = infer_df["X"].values
X_tensor = torch.tensor(np.stack(X_scaled), dtype=torch.float32, device=device)